# Demo: iBench FlightHotel

In the following, we will connect to a [Neo4j Community Edition](https://neo4j.com/product/neo4j-graph-database/) instance running inside a Docker container.
You can type 
```
sudo docker run --name neo4jDemo -p 7687:7687 -p 7474:7474 -v ~/research/DTGraph/output-ibench-data:/var/lib/neo4j/import --env=NEO4J_AUTH=none neo4j:5.16.0-community
``` 
to install and run a Neo4j Community Edition locally. (Of course you need to have [Docker](https://docs.docker.com/engine/install/)  already installed on your system.) You should then be able to access [Neo4j browser](http://localhost:7474/browser/) running locally on your computer.

You need to replace `~/research/DTGraph` with the DTGraph's installation path on your computer. We need to mount the volume on the Docker instance to run the import scripts.

*Note:* We have specifically tested the compatibility of this framework with Neo4j Community Edition 5.16.0, which was the latest versions by the time of writting this guide. 

In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
password = ""
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password=password)

For this tutorial, we will use the [FlightHotel](https://github.com/yannramusat/TPG/tree/main/input-ibench-config/flighthotel) data integration scenario from [iBench](https://github.com/RJMillerLab/ibench), which can be loaded into the database using the following command.

In [2]:
from dtgraph.scenarios.ibench_flighthotel import iBenchFlightHotel
iBenchFlightHotel.load(graph, size = 1_000)

Flushed database: Deleted 37588 nodes, deleted 29714 relationships, completed after 282 ms.
CSV:    Added 1000 labels, created 1000 nodes, set 3000 properties, created 0 relationships, completed after 159 ms.
CSV:    Added 1000 labels, created 1000 nodes, set 2000 properties, created 0 relationships, completed after 160 ms.


In [3]:
rule1 = Rule('''
MATCH (f:Flight), (h:Hotel)
WHERE f.fid = h.flid
GENERATE 
(l = (f.src):Location {
    name = f.src
})-[():FLIGHTS_TO]->(t = (f.src, f.dest):Travel {
    from = f.src,
    to = f.dest
}), 
(t)-[():FLIGHTS_TO]->(j = (f.dest):Location {
    name = f.dest
}),
(t)-[():HAS_HOTEL]->(m = (h.hid):Hotel2 {
    name = h.hid
})
''')

In [4]:
fh_transform = Transformation([rule1], with_diagnose = False)
fh_transform.apply_on(graph)

Index: Added 0 index, completed after 1 ms.
Rule: Added 5601 labels, created 2673 nodes, set 10669 properties, created 2996 relationships, completed after 300 ms.


300